In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras import Sequential
from keras.layers import Flatten, Dense, Rescaling, Conv2D, MaxPooling2D # Import necessary layers
import matplotlib.pyplot as plt
import os
import csv
from keras.utils import to_categorical
import kagglehub
import shutil
from tensorflow.keras.callbacks import EarlyStopping

from google.colab import drive
drive.mount("/content/drive")

# Download latest version
path = kagglehub.dataset_download("meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")

print("Path to dataset files:", path)

# Instead of removing the directory, let's check if it exists first:
dir_to_remove = os.path.join('/root/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1/Train',str(11))
if os.path.exists(dir_to_remove):
    shutil.rmtree(dir_to_remove)
    print(f"Removed directory: {dir_to_remove}")
else:
    print(f"Directory not found: {dir_to_remove}")

train_ds = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        validation_split=0.2,
        rotation_range=20,          # Randomly rotate images up to 20 degrees
        width_shift_range=0.2,      # Randomly shift images horizontally
        height_shift_range=0.2,     # Randomly shift images vertically
        shear_range=0.2,            # Randomly apply shearing transformations
        zoom_range=0.2,             # Randomly zoom in/out
        horizontal_flip=True,       # Randomly flip images horizontally
        fill_mode='nearest'         # Fill in missing pixels with nearest values
    )
train_data = '/root/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1/Train/'

# Create separate generators for training and validation
train = train_ds.flow_from_directory(train_data, target_size=(224, 224), color_mode='rgb', batch_size=16, class_mode='categorical', subset='training', shuffle=True)
validation = train_ds.flow_from_directory(train_data, target_size=(224, 224), color_mode='rgb', batch_size=16, class_mode='categorical', subset='validation', shuffle=False)

# Get the actual number of classes from the generator
num_classes = train.num_classes
print(f"Number of classes found: {num_classes}")

# Define the VGG16 model as the base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Create a new Sequential model
model = Sequential([
    base_model, # Add the VGG16 base model
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(num_classes, activation='softmax')
])


#The Adam algorithm computes the exponential moving average of the gradient (first moment)
#and the squared gradient (second moment) of the weights, where the parameters beta1 and beta2
#control the smoothing rates of these moving averages
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

#model.summary()
#An epoch is when all the training data is used at once and is defined as the total number of iterations
#of all the training data in one cycle for training the machine learning model.
# Increase epochs, but add early stopping
epochs = 20
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Reduce batch size if needed
train = train_ds.flow_from_directory(train_data, target_size=(224, 224), color_mode='rgb', batch_size=16, class_mode='categorical', subset='training', shuffle=True)
validation = train_ds.flow_from_directory(train_data, target_size=(224, 224), color_mode='rgb', batch_size=16, class_mode='categorical', subset='validation', shuffle=False)

# Use the validation generator in model.fit
history = model.fit(train, steps_per_epoch=20, epochs=epochs)

# Training loop with progress printing and early stopping
for epoch in range(epochs):
    history = model.fit(
        train,
        steps_per_epoch=20,  # Adjust if needed
        epochs=1,           # Train one epoch at a time
        validation_data=validation,
        callbacks=[early_stopping]
    )
    print(f"Epoch {epoch + 1}/{epochs} completed.")  # Progress update
    # Optionally print loss and accuracy here

Mounted at /content/drive


100%|██████████| 612M/612M [00:12<00:00, 50.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1
Removed directory: /root/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1/Train/11
Found 30312 images belonging to 42 classes.
Found 7577 images belonging to 42 classes.
Number of classes found: 42
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Found 30312 images belonging to 42 classes.
Found 7577 images belonging to 42 classes.
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/20 ━━━━━━━━━━━━━━━━━━━━ 19s 10s/step - accuracy: 0.0797 - loss: 52.5080

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(range(epochs), acc, label='Training Accuracy')
plt.plot(range(epochs), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(range(epochs), loss, label='Training Loss')
plt.plot(range(epochs), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Display the maximum validation accuracy
print("Maximum Validation Accuracy:", max(val_acc))